# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
fpath = 'Output/city_weather_data.csv'
city_weather_df = pd.read_csv(fpath).set_index('Unnamed: 0')
city_weather_df.head(10)


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
Unnamed: 0,,,,,,,,,
0,Hasaki,35.7333,140.8333,79.20,89,100,10.74,JP,1628357037
1,Bathsheba,13.2167,-59.5167,86.61,66,40,13.80,BB,1628356939
2,Narsaq,60.9167,-46.0500,52.02,79,100,4.97,GL,1628356949
3,Buta,2.7858,24.7300,70.20,99,99,2.80,CD,1628357329
4,Oistins,13.0667,-59.5333,86.09,66,40,13.80,BB,1628357330
5,Carnarvon,-24.8667,113.6333,66.27,90,90,8.05,AU,1628357330
6,Albany,42.6001,-73.9662,87.15,69,100,2.37,US,1628356836
7,Show Low,34.2542,-110.0298,83.71,33,20,5.75,US,1628357330
8,Jamestown,42.0970,-79.2353,83.30,63,75,3.00,US,1628356932


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [64]:
city_weather_df.dropna()
locations = city_weather_df[['Lat','Lng']].astype(float)
hm_humidity = city_weather_df[['Humidity']].astype(float)

hm_humidity

,Humidity
Unnamed: 0,
0,89.0
1,66.0
2,79.0
3,99.0
4,66.0
...,...
603,85.0
604,89.0
605,71.0


In [73]:
# Create a humidity Heatmap layer
figure_layout = {'width':'700px','height':'500px'}
fig = gmaps.figure(layout = figure_layout)

hm_humidity_layer = gmaps.heatmap_layer(locations, weights=hm_humidity.Humidity)

fig.add_layer(hm_humidity_layer)

fig

Figure(layout=FigureLayout(height='500px', width='700px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [52]:
hotel_df = city_weather_df[
    (city_weather_df['Max Temp']<=80) & (city_weather_df['Humidity']<40)
    & (city_weather_df['Wind Speed'] < 50) & (city_weather_df['Wind Speed'] < 10)
    & (city_weather_df['Cloudiness'] < 20)].dropna().reset_index(drop=True)
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Ürümqi,43.8010,87.6005,74.35,36,0,2.24,CN,1628357051
1,Oruzgan,32.8333,66.0000,78.30,13,0,2.44,AF,1628357344
2,Kang,-23.6752,22.7876,66.43,15,2,2.84,BW,1628357373
3,Chapais,49.7834,-74.8492,58.78,36,19,8.72,CA,1628357375
4,Masvingo,-20.0637,30.8277,63.27,32,0,6.49,ZW,1628357380


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [56]:
hotel_df['Hotel Name'] = 'None'

In [57]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
target_type = 'lodging'
target_radius = '5000'
target_lat = hotel_df['Lat']
target_lng = hotel_df['Lng']

hotel_list = []

count = 0
for l in hotel_df:    
    request_url = (f'{base_url}location={target_lat[count]},{target_lng[count]}&radius={target_radius}&key={g_key}')
    response = requests.get(request_url).json()
    hotel_list.append(response['results'][1]['name'])
    count += 1

hotel_df['Hotel Name'] = hotel_list

In [80]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Ürümqi,43.8010,87.6005,74.35,36,0,2.24,CN,1628357051,Jingu Hotel
1,Oruzgan,32.8333,66.0000,78.30,13,0,2.44,AF,1628357344,چوره
2,Kang,-23.6752,22.7876,66.43,15,2,2.84,BW,1628357373,Kang Ultra Stop
3,Chapais,49.7834,-74.8492,58.78,36,19,8.72,CA,1628357375,Hôtel Opémiska
4,Masvingo,-20.0637,30.8277,63.27,32,0,6.49,ZW,1628357380,Rujeko A
5,Shakawe,-18.3667,21.8500,66.90,16,0,4.29,BW,1628357471,PEP Shakawe
6,Pampas,-12.4000,-74.9000,63.54,34,3,8.01,PE,1628357474,MUNICIPALIDAD DE DANIEL HERNANDEZ
7,Qūchān,37.1060,58.5095,76.14,23,8,4.41,IR,1628357495,شبکه اینترنت دانشجویان
8,Luena,-11.7833,19.9167,62.94,16,13,4.54,AO,1628357519,La Paloma Restaurante
9,Venado Tuerto,-33.7456,-61.9688,76.77,32,0,8.79,AR,1628357531,Colegio de Abogados 3° Circunscripcion Judicia...


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [74]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [75]:
# Add marker layer ontop of heat map
hotel_marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(hotel_marker_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='500px', width='700px'))